In [4]:
import tensorflow as tf
import numpy as np
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    for file in os.listdir(file_dir_TC):
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    for file in os.listdir(file_dir_nonTC):
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return image_batch, label_batch
 

In [5]:
import tensorflow as tf
def inference(images, batch_size, n_classes):
 
 
    with tf.variable_scope('conv1') as scope:
        weights = tf.get_variable('weights',
                                  shape=[3, 3, 1, 16],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[16],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(images, weights, strides=[1, 1, 1, 1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
 
    with tf.variable_scope('pooling1_lrn') as scope:
            pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pooling1')
            norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
 
    with tf.variable_scope('conv2') as scope:
                weights = tf.get_variable('weights',
                                          shape=[3, 3, 16, 16],
                                          dtype=tf.float32,
                                          initializer=tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
                biases = tf.get_variable('biases',
                                         shape=[16],
                                         dtype=tf.float32,
                                         initializer=tf.constant_initializer(0.1))
                conv = tf.nn.conv2d(norm1, weights, strides=[1, 1, 1, 1], padding='SAME')
                pre_activation = tf.nn.bias_add(conv, biases)
                conv2 = tf.nn.relu(pre_activation, name='conv2')
 
    # pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME', name='pooling2')
 
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = tf.get_variable('weights',
                                  shape=[dim, 128],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[128],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
 
    # local4
    with tf.variable_scope('local4') as scope:
        weights = tf.get_variable('weights',
                                  shape=[128, 128],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[128],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')
 
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.get_variable('softmax_linear',
                                  shape=[128, n_classes],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[n_classes],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
 
    return softmax_linear
 
 
 
def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

def recall_precision(logits, labels, name):
    logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.arg_max(logits,1)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero((predict - 1) * labels)
        FP = tf.count_nonzero(predict * (labels - 1))
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall
# def evaluation(logits,labels,size):
#     with tf.variable_scope('result') as scope:
#         correct1=[tf.metrics.recall(labels=tf.cast(labels[i], tf.int32),predictions=tf.cast(logits[i],tf.int32)) for i in range(size)]
#         recall=tf.reduce_mean(correct1)
#         tf.summary.scalar(scope.name + '/recall', recall)
#         correct2=[tf.metrics.precision(labels=tf.cast(labels[i], tf.int32),predictions=tf.cast(logits[i],tf.int32)) for i in range(size)]
#         precision=tf.reduce_mean(correct2)
#         tf.summary.scalar(scope.name + '/precision', precision)
#     return recall,precision

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
N_CLASSES = 2 
IMG_W = 32  # resize
IMG_H = 32
BATCH_SIZE = 256
CAPACITY = 2000
MAX_STEP = 5000000 
learning_rate = 0.0001 
 
train_dir = './train_resnet/'
logs_train_dir = './train_resnet/'

file_dir_TC='/home/ubuntu/data/TC/'
file_dir_nonTC='/home/ubuntu/data/nonTC/'
# file_dir_valTC='/home/ubuntu/data/valTC/'
# file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

train, train_label = get_files(file_dir_TC,file_dir_nonTC,'TC','nonTC')
# valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
train_batch,train_label_batch=get_batch(train,
                                train_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

# valid_batch,valid_label_batch=get_batch(valid,
#                                 valid_label,
#                                 IMG_W,
#                                 IMG_H,
#                                 BATCH_SIZE,
#                                 CAPACITY)
# x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H,1])
# y = tf.placeholder(tf.float32, [BATCH_SIZE])
# y = tf.cast(y,tf.int64)

train_logits = inference(train_batch, BATCH_SIZE, N_CLASSES)
train_loss = losses(train_logits, train_label_batch, 'train')
# valid_loss = losses(train_logits, y, 'valid')
train_op = trainning(train_loss, learning_rate)
train__acc = evaluation(train_logits, train_label_batch, 'train')
# valid__acc = evaluation(train_logits, y, 'valid')
train_recall, train_precision = recall_precision(train_logits, train_label_batch, 'train')
# val_recall, val_precision = recall_precision(train_logits, y, 'valid')
summary_op = tf.summary.merge_all() 
 
sess = tf.Session()

train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
 

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
 
try:
    print('----------training start---------')
    for step in np.arange(MAX_STEP):
        
        if coord.should_stop():
                break
       
        _, tra_loss, tra_acc, tra_recall, tra_precision= sess.run([train_op, train_loss, train__acc, train_recall, train_precision])
        
        if step % 50 == 0:
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%, train recall = %.2f%%, train precision = %.2f%%' %(step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0))
            
#         if step % 200 == 0:    
#             val_loss, val_acc, cal_recall, val_precision = sess.run([valid_loss, valid__acc, val_recall, val_precision], feed_dict={x:valid_batch,y:valid_label_batch})
#             print('Step %d, valid loss = %.2f, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0))
            summary_str = sess.run(summary_op)
            train_writer.add_summary(summary_str, step)
      
        if step % 2000 == 0 or (step + 1) == MAX_STEP:
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')

finally:
    coord.request_stop()

There are 2016816 TC
There are 1737956 nonTC
----------training start---------
Step 0, train loss = 0.69, train accuracy = 45.70%, train recall = nan%, train precision = 0.00%
Step 50, train loss = 0.47, train accuracy = 78.12%, train recall = 98.21%, train precision = 38.73%
Step 100, train loss = 0.49, train accuracy = 76.17%, train recall = 89.71%, train precision = 41.78%
Step 150, train loss = 0.50, train accuracy = 76.95%, train recall = 94.12%, train precision = 37.80%
Step 200, train loss = 0.51, train accuracy = 75.00%, train recall = 90.57%, train precision = 34.53%
Step 250, train loss = 0.45, train accuracy = 81.64%, train recall = 88.17%, train precision = 59.42%
Step 300, train loss = 0.42, train accuracy = 81.64%, train recall = 92.93%, train precision = 60.13%
Step 350, train loss = 0.46, train accuracy = 77.34%, train recall = 91.57%, train precision = 53.52%
Step 400, train loss = 0.45, train accuracy = 79.69%, train recall = 91.36%, train precision = 52.11%
Step 450,

Step 4000, train loss = 0.38, train accuracy = 82.03%, train recall = 96.47%, train precision = 58.16%
Step 4050, train loss = 0.38, train accuracy = 81.25%, train recall = 94.67%, train precision = 55.04%
Step 4100, train loss = 0.34, train accuracy = 84.77%, train recall = 98.72%, train precision = 51.68%
Step 4150, train loss = 0.38, train accuracy = 81.64%, train recall = 96.15%, train precision = 56.39%
Step 4200, train loss = 0.30, train accuracy = 87.50%, train recall = 94.78%, train precision = 74.15%
Step 4250, train loss = 0.39, train accuracy = 80.08%, train recall = 95.12%, train precision = 54.17%
Step 4300, train loss = 0.37, train accuracy = 85.16%, train recall = 93.75%, train precision = 57.25%
Step 4350, train loss = 0.35, train accuracy = 83.98%, train recall = 93.41%, train precision = 60.71%
Step 4400, train loss = 0.33, train accuracy = 87.11%, train recall = 97.14%, train precision = 70.34%
Step 4450, train loss = 0.35, train accuracy = 83.20%, train recall = 96.

Step 8000, train loss = 0.35, train accuracy = 84.38%, train recall = 95.40%, train precision = 57.64%
Step 8050, train loss = 0.29, train accuracy = 91.02%, train recall = 94.57%, train precision = 67.44%
Step 8100, train loss = 0.32, train accuracy = 86.33%, train recall = 95.65%, train precision = 61.97%
Step 8150, train loss = 0.29, train accuracy = 87.50%, train recall = 98.88%, train precision = 65.19%
Step 8200, train loss = 0.28, train accuracy = 90.62%, train recall = 97.40%, train precision = 55.56%
Step 8250, train loss = 0.31, train accuracy = 86.72%, train recall = 97.62%, train precision = 62.12%
Step 8300, train loss = 0.41, train accuracy = 80.86%, train recall = 97.14%, train precision = 53.54%
Step 8350, train loss = 0.31, train accuracy = 88.67%, train recall = 94.32%, train precision = 61.48%
Step 8400, train loss = 0.32, train accuracy = 86.72%, train recall = 98.06%, train precision = 67.33%
Step 8450, train loss = 0.33, train accuracy = 87.50%, train recall = 97.

Step 12000, train loss = 0.32, train accuracy = 85.55%, train recall = 95.05%, train precision = 71.11%
Step 12050, train loss = 0.27, train accuracy = 89.06%, train recall = 94.25%, train precision = 63.57%
Step 12100, train loss = 0.31, train accuracy = 86.33%, train recall = 94.34%, train precision = 68.49%
Step 12150, train loss = 0.36, train accuracy = 85.16%, train recall = 93.75%, train precision = 68.18%
Step 12200, train loss = 0.31, train accuracy = 86.33%, train recall = 97.83%, train precision = 63.38%
Step 12250, train loss = 0.33, train accuracy = 83.59%, train recall = 97.44%, train precision = 58.02%
Step 12300, train loss = 0.33, train accuracy = 85.55%, train recall = 91.92%, train precision = 71.09%
Step 12350, train loss = 0.31, train accuracy = 87.50%, train recall = 91.89%, train precision = 73.91%
Step 12400, train loss = 0.34, train accuracy = 85.16%, train recall = 96.08%, train precision = 72.06%
Step 12450, train loss = 0.36, train accuracy = 83.59%, train re

Step 15950, train loss = 0.29, train accuracy = 87.50%, train recall = 92.71%, train precision = 68.99%
Step 16000, train loss = 0.26, train accuracy = 87.89%, train recall = 95.60%, train precision = 68.50%
Step 16050, train loss = 0.28, train accuracy = 89.84%, train recall = 96.81%, train precision = 65.47%
Step 16100, train loss = 0.25, train accuracy = 87.89%, train recall = 99.01%, train precision = 68.03%
Step 16150, train loss = 0.27, train accuracy = 88.67%, train recall = 98.94%, train precision = 71.54%
Step 16200, train loss = 0.25, train accuracy = 89.84%, train recall = 96.00%, train precision = 70.07%
Step 16250, train loss = 0.25, train accuracy = 87.11%, train recall = 97.89%, train precision = 72.09%
Step 16300, train loss = 0.25, train accuracy = 89.45%, train recall = 96.30%, train precision = 76.47%
Step 16350, train loss = 0.24, train accuracy = 89.84%, train recall = 97.06%, train precision = 72.26%
Step 16400, train loss = 0.30, train accuracy = 90.62%, train re

...

Step 114550, train loss = 0.22, train accuracy = 91.80%, train recall = 95.96%, train precision = 72.52%
Step 114600, train loss = 0.27, train accuracy = 87.11%, train recall = 96.26%, train precision = 72.03%
Step 114650, train loss = 0.23, train accuracy = 91.80%, train recall = 97.32%, train precision = 75.69%
Step 114700, train loss = 0.20, train accuracy = 92.19%, train recall = 99.03%, train precision = 71.83%
Step 114750, train loss = 0.28, train accuracy = 89.06%, train recall = 96.94%, train precision = 70.37%
Step 114800, train loss = 0.20, train accuracy = 91.80%, train recall = 98.89%, train precision = 72.95%
Step 114850, train loss = 0.20, train accuracy = 90.62%, train recall = 99.06%, train precision = 76.64%
Step 114900, train loss = 0.22, train accuracy = 91.02%, train recall = 97.70%, train precision = 72.03%
Step 114950, train loss = 0.18, train accuracy = 91.41%, train recall = 99.08%, train precision = 76.06%
Step 115000, train loss = 0.20, train accuracy = 92.19%

Step 118450, train loss = 0.31, train accuracy = 83.98%, train recall = 93.48%, train precision = 66.67%
Step 118500, train loss = 0.24, train accuracy = 88.28%, train recall = 96.88%, train precision = 69.40%
Step 118550, train loss = 0.21, train accuracy = 90.23%, train recall = 98.15%, train precision = 79.10%
Step 118600, train loss = 0.18, train accuracy = 92.19%, train recall = 97.22%, train precision = 72.41%
Step 118650, train loss = 0.25, train accuracy = 89.84%, train recall = 96.84%, train precision = 66.67%
Step 118700, train loss = 0.22, train accuracy = 89.06%, train recall = 98.98%, train precision = 71.85%
Step 118750, train loss = 0.26, train accuracy = 89.06%, train recall = 96.85%, train precision = 77.85%
Step 118800, train loss = 0.18, train accuracy = 92.19%, train recall = 100.00%, train precision = 75.89%
Step 118850, train loss = 0.20, train accuracy = 91.02%, train recall = 98.13%, train precision = 81.40%
Step 118900, train loss = 0.21, train accuracy = 90.62

Step 122350, train loss = 0.18, train accuracy = 92.19%, train recall = 97.62%, train precision = 82.55%
Step 122400, train loss = 0.24, train accuracy = 89.84%, train recall = 96.77%, train precision = 75.63%
Step 122450, train loss = 0.17, train accuracy = 93.36%, train recall = 99.07%, train precision = 77.94%
Step 122500, train loss = 0.19, train accuracy = 90.62%, train recall = 98.02%, train precision = 70.21%
Step 122550, train loss = 0.19, train accuracy = 93.36%, train recall = 99.02%, train precision = 70.14%
Step 122600, train loss = 0.23, train accuracy = 90.62%, train recall = 98.11%, train precision = 75.36%
Step 122650, train loss = 0.22, train accuracy = 89.45%, train recall = 97.17%, train precision = 72.03%
Step 122700, train loss = 0.24, train accuracy = 90.23%, train recall = 94.78%, train precision = 80.74%
Step 122750, train loss = 0.23, train accuracy = 91.80%, train recall = 94.40%, train precision = 80.82%
Step 122800, train loss = 0.22, train accuracy = 89.84%

Step 126250, train loss = 0.20, train accuracy = 92.19%, train recall = 95.90%, train precision = 86.03%
Step 126300, train loss = 0.19, train accuracy = 91.02%, train recall = 98.26%, train precision = 80.71%
Step 126350, train loss = 0.21, train accuracy = 92.97%, train recall = 96.08%, train precision = 77.17%
Step 126400, train loss = 0.21, train accuracy = 88.67%, train recall = 96.72%, train precision = 79.73%
Step 126450, train loss = 0.22, train accuracy = 90.23%, train recall = 96.88%, train precision = 71.54%
Step 126500, train loss = 0.17, train accuracy = 92.19%, train recall = 100.00%, train precision = 83.70%
Step 126550, train loss = 0.18, train accuracy = 93.36%, train recall = 97.39%, train precision = 85.50%
Step 126600, train loss = 0.20, train accuracy = 92.58%, train recall = 99.02%, train precision = 72.14%
Step 126650, train loss = 0.19, train accuracy = 93.36%, train recall = 97.30%, train precision = 80.00%
Step 126700, train loss = 0.27, train accuracy = 89.45

Step 130150, train loss = 0.16, train accuracy = 94.92%, train recall = 98.20%, train precision = 81.34%
Step 130200, train loss = 0.23, train accuracy = 88.67%, train recall = 97.94%, train precision = 73.08%
Step 130250, train loss = 0.24, train accuracy = 90.23%, train recall = 97.85%, train precision = 67.91%
Step 130300, train loss = 0.13, train accuracy = 95.31%, train recall = 99.13%, train precision = 80.85%
Step 130350, train loss = 0.22, train accuracy = 91.02%, train recall = 97.09%, train precision = 79.37%
Step 130400, train loss = 0.20, train accuracy = 91.02%, train recall = 95.83%, train precision = 84.56%
Step 130450, train loss = 0.21, train accuracy = 91.02%, train recall = 98.10%, train precision = 79.84%
Step 130500, train loss = 0.20, train accuracy = 91.41%, train recall = 97.54%, train precision = 80.41%
Step 130550, train loss = 0.27, train accuracy = 89.45%, train recall = 97.09%, train precision = 70.42%
Step 130600, train loss = 0.17, train accuracy = 92.97%